In [19]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [20]:
transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [21]:
train_data = torchvision.datasets.ImageFolder(root='data/CatDogDataset/train', transform=transforms)
val_data = torchvision.datasets.ImageFolder(root='data/CatDogDataset/val', transform=transforms)
test_data = torchvision.datasets.ImageFolder(root='data/CatDogDataset/test', transform=transforms)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=False)    
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

In [22]:
image, label = train_data[0]

In [23]:
image.size()  # Kích thước của ảnh sau khi chuyển đổi

torch.Size([3, 256, 256])

In [24]:
class_names = train_data.classes
print(class_names)  # ['Cat', 'Dog']

['Cat', 'Dog']


In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)  # Lớp convolutional đầu tiên
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)  # Lớp pooling đầu tiên

        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # Lớp convolutional thứ hai
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)  # Lớp pooling thứ hai
        
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)  # Lớp convolutional thứ ba
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)  # Lớp pooling thứ ba

        self.fc1 = nn.Linear(128 * 32 * 32, 256)  # Lớp fully connected
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, 2)


    def forward(self, x):
        x = self.pool1(F.relu(self.batchnorm1(self.conv1(x))))
        x = self.pool2(F.relu(self.batchnorm2(self.conv2(x))))
        x = self.pool3(F.relu(self.batchnorm3(self.conv3(x))))

        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNet().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(10):
    model.train()
    total_loss, correct, total = 0, 0, 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/10], Loss: {total_loss/len(train_loader):.4f}, Train Acc: {acc:.2f}%")


c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch [1/10], Loss: 1.5026, Train Acc: 62.37%
Epoch [2/10], Loss: 0.5786, Train Acc: 68.47%
Epoch [3/10], Loss: 0.5598, Train Acc: 70.24%
Epoch [4/10], Loss: 0.5333, Train Acc: 72.35%
Epoch [5/10], Loss: 0.5187, Train Acc: 73.36%
Epoch [6/10], Loss: 0.4888, Train Acc: 75.64%
Epoch [7/10], Loss: 0.4612, Train Acc: 77.00%
Epoch [8/10], Loss: 0.4447, Train Acc: 78.16%


In [ ]:
torch.save(model.state_dict(), 'cat_dog_model.pth')